# **Comitê de Classificadores**
---
---

In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


In [2]:
hog_128_16 = pd.read_csv('../datasets/hog_128_16.csv')
hog_128_20 = pd.read_csv('../datasets/hog_128_20.csv')
cnn_VGG16_AVG_128 = pd.read_csv('../datasets/cnn_VGG16_AVG_128.csv')
cnn_VGG19_AVG_128 = pd.read_csv('../datasets/cnn_VGG19_AVG_128.csv')
cnn_VGG16_MAX_128 = pd.read_csv('../datasets/cnn_VGG16_MAX_128.csv')
cnn_VGG19_MAX_128 = pd.read_csv('../datasets/cnn_VGG19_MAX_128.csv')
hog_128_16_PCA = pd.read_csv('../datasets/hog_128_16_PCA.csv')
hog_128_20_PCA = pd.read_csv('../datasets/hog_128_20_PCA.csv')
cnn_VGG16_AVG_128_PCA = pd.read_csv('../datasets/cnn_VGG16_AVG_128_PCA.csv')
cnn_VGG19_AVG_128_PCA= pd.read_csv('../datasets/cnn_VGG19_AVG_128_PCA.csv')
cnn_VGG16_MAX_128_PCA= pd.read_csv('../datasets/cnn_VGG16_MAX_128_PCA.csv')
cnn_VGG19_MAX_128_PCA = pd.read_csv('../datasets/cnn_VGG19_MAX_128_PCA.csv')

In [3]:
#Instanciando um dict com todos os datasets
dataframes = {
    'hog_128_16': hog_128_16,
    'hog_128_20': hog_128_20,
    'cnn_VGG16_AVG_128': cnn_VGG16_AVG_128,
    'cnn_VGG19_AVG_128': cnn_VGG19_AVG_128,
    'cnn_VGG16_MAX_128': cnn_VGG16_MAX_128,
    'cnn_VGG19_MAX_128': cnn_VGG19_MAX_128,
    'hog_128_16_PCA': hog_128_16_PCA,
    'hog_128_20_PCA': hog_128_20_PCA,
    'cnn_VGG16_AVG_128_PCA': cnn_VGG16_AVG_128_PCA,
    'cnn_VGG19_AVG_128_PCA': cnn_VGG19_AVG_128_PCA,
    'cnn_VGG16_MAX_128_PCA': cnn_VGG16_MAX_128_PCA,
    'cnn_VGG19_MAX_128_PCA': cnn_VGG19_MAX_128_PCA
}

## **1.Bagging**

### 1.1.Bagging Padrão

Vamos realizar bagging utilizando os melhores parâmetros que encontramos nas tarefas anteriores.
* K-NN: k=6
* Decision Tree: max_depth=8
* NaiveBayes: complement_nb
* MLP: activation= 'relu', hidden_layer_sizes= (884, 883), learning_rate_init= 0.001, max_iter= 5000, solver= 'sgd'

In [4]:
# Configurações dos classificadores
classifiers = {
    'k-NN': KNeighborsClassifier(n_neighbors=6),
    'AD': DecisionTreeClassifier(max_depth=8),
    'NB': ComplementNB(),
    'MLP': MLPClassifier(
        activation='relu',
        hidden_layer_sizes=(884, 883),
        learning_rate_init=0.001,
        max_iter=5000,
        solver='sgd'
    )
}

# Valores para Bagging
n_estimators_list = [10, 20, 30]

# Dicionário final para armazenar resultados
results = {}


In [8]:
# Loop sobre os dataframes
for name, df in dataframes.items():
    print(f"Iniciando processamento do DataFrame: {name}")
    
    X = df.drop(columns=['label'])
    y = df['label']
    
    # Resultados intermediários para cada dataframe
    df_results = []
    
    # Holdout (70/30)
    print(f"Realizando Holdout 70/30 para o DataFrame: {name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    
    # Avaliação dos classificadores no Holdout
    holdout_results = {}
    for clf_name, clf in classifiers.items():
        print(f"  Holdout: Avaliando classificador {clf_name} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    Holdout: Bagging com {n_estimators} estimadores para {clf_name}")
            bagging = BaggingClassifier(estimator=clf, n_estimators=n_estimators, random_state=42)
            
            # Tratamento para ComplementNB
            if clf_name == "NB":
                scaler = MinMaxScaler()
                X_train_transformed = scaler.fit_transform(X_train)
                X_test_transformed = scaler.transform(X_test)
                bagging.fit(X_train_transformed, y_train)
                accuracy = bagging.score(X_test_transformed, y_test)
            else:
                bagging.fit(X_train, y_train)
                accuracy = bagging.score(X_test, y_test)

            holdout_results[(clf_name, n_estimators)] = accuracy
    
    # K-Fold Cross-Validation
    print(f"Realizando K-Fold Cross-Validation para o DataFrame: {name}")
    kfold_results = {}
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for clf_name, clf in classifiers.items():
        print(f"  K-Fold: Avaliando classificador {clf_name} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    K-Fold: Bagging com {n_estimators} estimadores para {clf_name}")
            bagging = BaggingClassifier(estimator=clf, n_estimators=n_estimators, random_state=42)
            
            if clf_name == "NB":
                scaler = MinMaxScaler()
                X_transformed = scaler.fit_transform(X)
                scores = cross_val_score(bagging, X_transformed, y, cv=skf, scoring='accuracy')
            else:
                scores = cross_val_score(bagging, X, y, cv=skf, scoring='accuracy')
            
            kfold_results[(clf_name, n_estimators)] = scores.mean()
    
    # Montando DataFrame para o MultiIndex
    for method, acc_dict in zip(['70/30', '10-fold CV'], [holdout_results, kfold_results]):
        rows = []
        for (clf_name, n_estimators), accuracy in acc_dict.items():
            rows.append((method, clf_name, n_estimators, accuracy))
        
        df_temp = pd.DataFrame(rows, columns=['Method', 'Classifier', 'n_estimators', 'Accuracy'])
        df_results.append(df_temp)
    
    # Concatenar resultados
    df_combined = pd.concat(df_results)
    df_combined['Media (Acc TAM)'] = df_combined.groupby(['Method', 'Classifier'])['Accuracy'].transform('mean')
    results[name] = df_combined
    print(f"Finalizado o processamento do DataFrame: {name}")

# Criar MultiIndex final
final_result = pd.DataFrame()
for name, df in results.items():
    multiindex = pd.MultiIndex.from_tuples(
        [(name, row['Method'], row['Classifier']) for _, row in df.iterrows()],
        names=['DataFrame', 'Split Method', 'Classifier']
    )
    df.index = multiindex
    df = df.drop(columns=['Method', 'Classifier'])
    final_result = pd.concat([final_result, df])

Iniciando processamento do DataFrame: hog_128_16
Realizando Holdout 70/30 para o DataFrame: hog_128_16
  Holdout: Avaliando classificador k-NN no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para k-NN
    Holdout: Bagging com 20 estimadores para k-NN
    Holdout: Bagging com 30 estimadores para k-NN
  Holdout: Avaliando classificador AD no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para AD
    Holdout: Bagging com 20 estimadores para AD
    Holdout: Bagging com 30 estimadores para AD
  Holdout: Avaliando classificador NB no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para NB
    Holdout: Bagging com 20 estimadores para NB
    Holdout: Bagging com 30 estimadores para NB
  Holdout: Avaliando classificador MLP no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para MLP
    Holdout: Bagging com 20 estimadores para MLP
    Holdout: Bagging com 30 estimadores para MLP
Realizando K-Fold Cross-Validation para o DataFrame: hog_128_

In [9]:
bagging_acc_1 = final_result

In [29]:
# Ajustar o DataFrame para transformar n_estimators em colunas
final_result_reset = final_result.reset_index()  # Redefine o índice para facilitar a manipulação
pivot_result = final_result_reset.pivot_table(
    index=['DataFrame', 'Split Method', 'Classifier'],
    columns='n_estimators',
    values='Accuracy'
)

# Adicionar a coluna de média ('Média (Acc TAM)')
pivot_result['Média (Acc TAM)'] = pivot_result.mean(axis=1)

# Salvar o DataFrame ajustado em Excel
pivot_result.to_excel('resultados_bagging_ajustado.xlsx', sheet_name='Resultados', index=True)

### 1.2.Bagging com max_features

In [33]:
# Loop sobre os dataframes
for name, df in dataframes.items():
    print(f"Iniciando processamento do DataFrame: {name}")
    
    X = df.drop(columns=['label'])
    y = df['label']
    
    # Resultados intermediários para cada dataframe
    df_results = []
    
    # Holdout (70/30)
    print(f"Realizando Holdout 70/30 para o DataFrame: {name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    
    # Avaliação dos classificadores no Holdout
    holdout_results = {}
    for clf_name, clf in classifiers.items():
        print(f"  Holdout: Avaliando classificador {clf_name} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    Holdout: Bagging com {n_estimators} estimadores para {clf_name}")
            bagging = BaggingClassifier(
                estimator=clf,
                n_estimators=n_estimators,
                max_features=0.5,  # Ajuste para max_features = 0.5
                random_state=42
            )
            
            # Tratamento para ComplementNB
            if clf_name == "NB":
                scaler = MinMaxScaler()
                X_train_transformed = scaler.fit_transform(X_train)
                X_test_transformed = scaler.transform(X_test)
                bagging.fit(X_train_transformed, y_train)
                accuracy = bagging.score(X_test_transformed, y_test)
            else:
                bagging.fit(X_train, y_train)
                accuracy = bagging.score(X_test, y_test)

            holdout_results[(clf_name, n_estimators)] = accuracy
    
    # K-Fold Cross-Validation
    print(f"Realizando K-Fold Cross-Validation para o DataFrame: {name}")
    kfold_results = {}
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for clf_name, clf in classifiers.items():
        print(f"  K-Fold: Avaliando classificador {clf_name} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    K-Fold: Bagging com {n_estimators} estimadores para {clf_name}")
            bagging = BaggingClassifier(
                estimator=clf,
                n_estimators=n_estimators,
                max_features=0.5,  # Ajuste para max_features = 0.5
                random_state=42
            )
            
            if clf_name == "NB":
                scaler = MinMaxScaler()
                X_transformed = scaler.fit_transform(X)
                scores = cross_val_score(bagging, X_transformed, y, cv=skf, scoring='accuracy')
            else:
                scores = cross_val_score(bagging, X, y, cv=skf, scoring='accuracy')
            
            kfold_results[(clf_name, n_estimators)] = scores.mean()
    
    # Montando DataFrame para o MultiIndex
    for method, acc_dict in zip(['70/30', '10-fold CV'], [holdout_results, kfold_results]):
        rows = []
        for (clf_name, n_estimators), accuracy in acc_dict.items():
            rows.append((method, clf_name, n_estimators, accuracy))
        
        df_temp = pd.DataFrame(rows, columns=['Method', 'Classifier', 'n_estimators', 'Accuracy'])
        df_results.append(df_temp)
    
    # Concatenar resultados
    df_combined = pd.concat(df_results)
    df_combined['Media (Acc TAM)'] = df_combined.groupby(['Method', 'Classifier'])['Accuracy'].transform('mean')
    results[name] = df_combined
    print(f"Finalizado o processamento do DataFrame: {name}")

# Criar MultiIndex final
final_result = pd.DataFrame()
for name, df in results.items():
    multiindex = pd.MultiIndex.from_tuples(
        [(name, row['Method'], row['Classifier']) for _, row in df.iterrows()],
        names=['DataFrame', 'Split Method', 'Classifier']
    )
    df.index = multiindex
    df = df.drop(columns=['Method', 'Classifier'])
    final_result = pd.concat([final_result, df])


Iniciando processamento do DataFrame: hog_128_16
Realizando Holdout 70/30 para o DataFrame: hog_128_16
  Holdout: Avaliando classificador k-NN no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para k-NN
    Holdout: Bagging com 20 estimadores para k-NN
    Holdout: Bagging com 30 estimadores para k-NN
  Holdout: Avaliando classificador AD no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para AD
    Holdout: Bagging com 20 estimadores para AD
    Holdout: Bagging com 30 estimadores para AD
  Holdout: Avaliando classificador NB no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para NB
    Holdout: Bagging com 20 estimadores para NB
    Holdout: Bagging com 30 estimadores para NB
  Holdout: Avaliando classificador MLP no DataFrame hog_128_16
    Holdout: Bagging com 10 estimadores para MLP
    Holdout: Bagging com 20 estimadores para MLP
    Holdout: Bagging com 30 estimadores para MLP
Realizando K-Fold Cross-Validation para o DataFrame: hog_128_

In [34]:
bagging_acc_2 = final_result

In [35]:
# Ajustar o DataFrame para transformar n_estimators em colunas
final_result_reset = final_result.reset_index()  # Redefine o índice para facilitar a manipulação
pivot_result = final_result_reset.pivot_table(
    index=['DataFrame', 'Split Method', 'Classifier'],
    columns='n_estimators',
    values='Accuracy'
)

# Adicionar a coluna de média ('Média (Acc TAM)')
pivot_result['Média (Acc TAM)'] = pivot_result.mean(axis=1)

# Salvar o DataFrame ajustado em Excel
pivot_result.to_excel('resultados_bagging_ajustado_2.xlsx', sheet_name='Resultados', index=True)

## **2.Boosting**

In [42]:
# Parâmetros
n_estimators_list = [10, 20, 30]
results = {}

# Loop sobre os dataframes
for name, df in dataframes.items():
    print(f"Iniciando processamento do DataFrame: {name}")
    
    X = df.drop(columns=['label'])
    y = df['label']
    
    # Resultados intermediários para cada dataframe
    df_results = []
    
    # Holdout (70/30)
    print(f"Realizando Holdout 70/30 para o DataFrame: {name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    
    # Avaliação dos classificadores no Holdout
    holdout_results = {}
    classifiers = {
        'AD': DecisionTreeClassifier(max_depth=8, random_state=42),
        'NB': ComplementNB()
    }
    for clf_name, clf in classifiers.items():
        print(f"  Holdout: Avaliando classificador {clf_name} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    Holdout: Boosting com {n_estimators} estimadores para {clf_name}")
            boosting = AdaBoostClassifier(
                estimator=clf,
                n_estimators=n_estimators,
                random_state=42
            )
            
            # Tratamento para ComplementNB
            if clf_name == "NB":
                scaler = MinMaxScaler()
                X_train_transformed = scaler.fit_transform(X_train)
                X_test_transformed = scaler.transform(X_test)
                boosting.fit(X_train_transformed, y_train)
                accuracy = boosting.score(X_test_transformed, y_test)
            else:
                boosting.fit(X_train, y_train)
                accuracy = boosting.score(X_test, y_test)

            holdout_results[(clf_name, n_estimators)] = accuracy
    
    # K-Fold Cross-Validation
    print(f"Realizando K-Fold Cross-Validation para o DataFrame: {name}")
    kfold_results = {}
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for clf_name, clf in classifiers.items():
        print(f"  K-Fold: Avaliando classificador {clf_name} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    K-Fold: Boosting com {n_estimators} estimadores para {clf_name}")
            boosting = AdaBoostClassifier(
                estimator=clf,
                n_estimators=n_estimators,
                random_state=42
            )
            
            if clf_name == "NB":
                scaler = MinMaxScaler()
                X_transformed = scaler.fit_transform(X)
                scores = cross_val_score(boosting, X_transformed, y, cv=skf, scoring='accuracy')
            else:
                scores = cross_val_score(boosting, X, y, cv=skf, scoring='accuracy')
            
            kfold_results[(clf_name, n_estimators)] = scores.mean()
    
    # Montando DataFrame para o MultiIndex
    for method, acc_dict in zip(['70/30', '10-fold CV'], [holdout_results, kfold_results]):
        rows = []
        for (clf_name, n_estimators), accuracy in acc_dict.items():
            rows.append((method, clf_name, n_estimators, accuracy))
        
        df_temp = pd.DataFrame(rows, columns=['Method', 'Classifier', 'n_estimators', 'Accuracy'])
        df_results.append(df_temp)
    
    # Concatenar resultados
    df_combined = pd.concat(df_results)
    df_combined['Media (Acc TAM)'] = df_combined.groupby(['Method', 'Classifier'])['Accuracy'].transform('mean')
    results[name] = df_combined
    print(f"Finalizado o processamento do DataFrame: {name}")

# Criar MultiIndex final
final_result = pd.DataFrame()
for name, df in results.items():
    multiindex = pd.MultiIndex.from_tuples(
        [(name, row['Method'], row['Classifier']) for _, row in df.iterrows()],
        names=['DataFrame', 'Split Method', 'Classifier']
    )
    df.index = multiindex
    df = df.pivot(columns='n_estimators', values=['Accuracy'])
    df['Média (Acc TAM)'] = df.mean(axis=1)  # Calcula a média da linha
    final_result = pd.concat([final_result, df])

# Salvar o resultado final em Excel
final_result.to_excel('resultados_boosting_ajustado_final_com_media.xlsx', sheet_name='Resultados', index=True)

print("Processamento completo! Resultados salvos no arquivo 'resultados_boosting_ajustado_final_com_media.xlsx'.")

Iniciando processamento do DataFrame: hog_128_16
Realizando Holdout 70/30 para o DataFrame: hog_128_16
  Holdout: Avaliando classificador AD no DataFrame hog_128_16
    Holdout: Boosting com 10 estimadores para AD


c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


    Holdout: Boosting com 20 estimadores para AD


c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


    Holdout: Boosting com 30 estimadores para AD


...

  K-Fold: Avaliando classificador NB no DataFrame cnn_VGG19_MAX_128_PCA
    K-Fold: Boosting com 10 estimadores para NB
    K-Fold: Boosting com 20 estimadores para NB


c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\dav

    K-Fold: Boosting com 30 estimadores para NB


c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\dav

Finalizado o processamento do DataFrame: cnn_VGG19_MAX_128_PCA
Processamento completo! Resultados salvos no arquivo 'resultados_boosting_ajustado_final_com_media.xlsx'.


## **3.Random Forest**

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import pandas as pd

# Parâmetros
n_estimators_list = [10, 20, 30, 100]
criteria = ['gini', 'entropy', 'log_loss']
results = {}

# Loop sobre os dataframes
for name, df in dataframes.items():
    print(f"Iniciando processamento do DataFrame: {name}")
    
    X = df.drop(columns=['label'])
    y = df['label']
    
    # Resultados intermediários para cada dataframe
    df_results = []
    
    # Holdout (70/30)
    print(f"Realizando Holdout 70/30 para o DataFrame: {name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    
    # Avaliação no Holdout
    holdout_results = {}
    for criterion in criteria:
        print(f"  Holdout: Avaliando estratégia {criterion} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    Holdout: Random Forest com {n_estimators} estimadores e {criterion} criterion")
            rf = RandomForestClassifier(
                n_estimators=n_estimators,
                criterion=criterion,
                random_state=42
            )
            
            rf.fit(X_train, y_train)
            accuracy = rf.score(X_test, y_test)
            holdout_results[(criterion, n_estimators)] = accuracy
    
    # K-Fold Cross-Validation
    print(f"Realizando K-Fold Cross-Validation para o DataFrame: {name}")
    kfold_results = {}
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for criterion in criteria:
        print(f"  K-Fold: Avaliando estratégia {criterion} no DataFrame {name}")
        for n_estimators in n_estimators_list:
            print(f"    K-Fold: Random Forest com {n_estimators} estimadores e {criterion} criterion")
            rf = RandomForestClassifier(
                n_estimators=n_estimators,
                criterion=criterion,
                random_state=42
            )
            
            scores = cross_val_score(rf, X, y, cv=skf, scoring='accuracy')
            kfold_results[(criterion, n_estimators)] = scores.mean()
    
    # Montando DataFrame para o MultiIndex
    for method, acc_dict in zip(['70/30', '10-fold CV'], [holdout_results, kfold_results]):
        rows = []
        for (criterion, n_estimators), accuracy in acc_dict.items():
            rows.append((method, criterion, n_estimators, accuracy))
        
        df_temp = pd.DataFrame(rows, columns=['Method', 'Criterion', 'n_estimators', 'Accuracy'])
        df_results.append(df_temp)
    
    # Concatenar resultados
    df_combined = pd.concat(df_results)
    df_combined['Media (Acc TAM)'] = df_combined.groupby(['Method', 'Criterion'])['Accuracy'].transform('mean')
    results[name] = df_combined
    print(f"Finalizado o processamento do DataFrame: {name}")

# Criar MultiIndex final
final_result = pd.DataFrame()
for name, df in results.items():
    multiindex = pd.MultiIndex.from_tuples(
        [(name, row['Method'], row['Criterion']) for _, row in df.iterrows()],
        names=['DataFrame', 'Split Method', 'Criterion']
    )
    df.index = multiindex
    df = df.pivot(columns='n_estimators', values='Accuracy')
    df['Média (Acc TAM)'] = df.mean(axis=1)  # Calcula a média da linha
    final_result = pd.concat([final_result, df])

# Salvar o resultado final em Excel
final_result.to_excel('resultados_random_forest_ajustado.xlsx', sheet_name='Resultados', index=True)

print("Processamento completo! Resultados salvos no arquivo 'resultados_random_forest_ajustado.xlsx'.")


Iniciando processamento do DataFrame: hog_128_16
Realizando Holdout 70/30 para o DataFrame: hog_128_16
  Holdout: Avaliando estratégia gini no DataFrame hog_128_16
    Holdout: Random Forest com 10 estimadores e gini criterion
    Holdout: Random Forest com 20 estimadores e gini criterion
    Holdout: Random Forest com 30 estimadores e gini criterion
    Holdout: Random Forest com 100 estimadores e gini criterion
  Holdout: Avaliando estratégia entropy no DataFrame hog_128_16
    Holdout: Random Forest com 10 estimadores e entropy criterion
    Holdout: Random Forest com 20 estimadores e entropy criterion
    Holdout: Random Forest com 30 estimadores e entropy criterion
    Holdout: Random Forest com 100 estimadores e entropy criterion
  Holdout: Avaliando estratégia log_loss no DataFrame hog_128_16
    Holdout: Random Forest com 10 estimadores e log_loss criterion
    Holdout: Random Forest com 20 estimadores e log_loss criterion
    Holdout: Random Forest com 30 estimadores e log_los

## **4.Stacking**

In [53]:
# Redefinindo o ComplementNB com Pipeline
classifiers = {
    'k-NN': KNeighborsClassifier(),
    'DT': DecisionTreeClassifier(random_state=42),
    'NB': make_pipeline(MinMaxScaler(), ComplementNB()),  # Pipeline para escalar os dados
    'MLP': MLPClassifier(hidden_layer_sizes=(50,), max_iter=500, random_state=42)  # Configuração simples do MLP
}

# Loop sobre os dataframes
for name, df in dataframes.items():
    print(f"Iniciando processamento do DataFrame: {name}")
    
    X = df.drop(columns=['label'])
    y = df['label']
    
    # Resultados intermediários para cada dataframe
    df_results = []
    
    # Holdout (70/30)
    print(f"Realizando Holdout 70/30 para o DataFrame: {name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    
    # Avaliação dos classificadores no Holdout
    holdout_results = {}
    for n_classifiers in n_classifiers_list:
        print(f"    Holdout: Stacking com {n_classifiers} classificadores base para {name}")
        
        # Criar os estimadores base
        estimators = [(clf_name, clf) for clf_name, clf in classifiers.items()][:n_classifiers]
        
        # Criar o modelo de Stacking
        stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
        
        # Treinar e avaliar
        stacking.fit(X_train, y_train)
        accuracy = stacking.score(X_test, y_test)
        
        holdout_results[n_classifiers] = accuracy
    
    # K-Fold Cross-Validation
    print(f"Realizando K-Fold Cross-Validation para o DataFrame: {name}")
    kfold_results = {}
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for n_classifiers in n_classifiers_list:
        print(f"    K-Fold: Stacking com {n_classifiers} classificadores base para {name}")
        
        estimators = [(clf_name, clf) for clf_name, clf in classifiers.items()][:n_classifiers]
        stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
        
        scores = cross_val_score(stacking, X, y, cv=skf, scoring='accuracy')
        kfold_results[n_classifiers] = scores.mean()
    
    # Montando DataFrame para o MultiIndex
    for method, acc_dict in zip(split_methods, [holdout_results, kfold_results]):
        rows = []
        for n_classifiers, accuracy in acc_dict.items():
            rows.append((method, n_classifiers, accuracy))
        
        df_temp = pd.DataFrame(rows, columns=['Method', 'n_classifiers', 'Accuracy'])
        df_results.append(df_temp)
    
    # Concatenar resultados
    df_combined = pd.concat(df_results)
    df_combined['Média'] = df_combined.groupby(['Method', 'n_classifiers'])['Accuracy'].transform('mean')
    results[name] = df_combined
    print(f"Finalizado o processamento do DataFrame: {name}")

# Criar MultiIndex final
final_result = pd.DataFrame()
for name, df in results.items():
    multiindex = pd.MultiIndex.from_tuples(
        [(name, row['Method'], row['n_classifiers']) for _, row in df.iterrows()],
        names=['DataFrame', 'Split Method', 'n_classifiers']
    )
    df.index = multiindex
    df = df.drop(columns=['Method', 'n_classifiers'])
    final_result = pd.concat([final_result, df])

# Salvar o resultado final em Excel
final_result.to_excel('stacking_heterogeneo_resultados.xlsx', sheet_name='Resultados', index=True)

Iniciando processamento do DataFrame: hog_128_16
Realizando Holdout 70/30 para o DataFrame: hog_128_16
    Holdout: Stacking com 5 classificadores base para hog_128_16


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


    Holdout: Stacking com 10 classificadores base para hog_128_16


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


    Holdout: Stacking com 15 classificadores base para hog_128_16


...

c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

    K-Fold: Stacking com 20 classificadores base para cnn_VGG19_MAX_128_PCA


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

Finalizado o processamento do DataFrame: cnn_VGG19_MAX_128_PCA


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


## **6.Voting**

In [52]:
from sklearn.ensemble import VotingClassifier

# Configuração dos classificadores base
classifiers = {
    'k-NN': KNeighborsClassifier(),
    'DT': DecisionTreeClassifier(random_state=42),
    'NB': make_pipeline(MinMaxScaler(), ComplementNB()),  # Pipeline para escalar os dados
    'MLP': MLPClassifier(hidden_layer_sizes=(50,), max_iter=500, random_state=42)  # Configuração simples do MLP
}

# Variáveis de configuração
n_classifiers_list = [5, 10, 15, 20]  # Número de classificadores base
split_methods = ['70/30', '10-fold CV']

# Resultados
results = {}

# Loop sobre os dataframes
for name, df in dataframes.items():
    print(f"Iniciando processamento do DataFrame: {name}")
    
    X = df.drop(columns=['label'])
    y = df['label']
    
    # Resultados intermediários para cada dataframe
    df_results = []
    
    # Holdout (70/30)
    print(f"Realizando Holdout 70/30 para o DataFrame: {name}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
    
    # Avaliação dos classificadores no Holdout
    holdout_results = {}
    for n_classifiers in n_classifiers_list:
        print(f"    Holdout: Voting com {n_classifiers} classificadores base para {name}")
        
        # Criar os estimadores base
        estimators = [(clf_name, clf) for clf_name, clf in classifiers.items()][:n_classifiers]
        
        # Criar o modelo de Voting
        voting = VotingClassifier(estimators=estimators, voting='soft')
        
        # Treinar e avaliar
        voting.fit(X_train, y_train)
        accuracy = voting.score(X_test, y_test)
        
        holdout_results[n_classifiers] = accuracy
    
    # K-Fold Cross-Validation
    print(f"Realizando K-Fold Cross-Validation para o DataFrame: {name}")
    kfold_results = {}
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for n_classifiers in n_classifiers_list:
        print(f"    K-Fold: Voting com {n_classifiers} classificadores base para {name}")
        
        estimators = [(clf_name, clf) for clf_name, clf in classifiers.items()][:n_classifiers]
        voting = VotingClassifier(estimators=estimators, voting='soft')
        
        scores = cross_val_score(voting, X, y, cv=skf, scoring='accuracy')
        kfold_results[n_classifiers] = scores.mean()
    
    # Montando DataFrame para o MultiIndex
    for method, acc_dict in zip(split_methods, [holdout_results, kfold_results]):
        rows = []
        for n_classifiers, accuracy in acc_dict.items():
            rows.append((method, n_classifiers, accuracy))
        
        df_temp = pd.DataFrame(rows, columns=['Method', 'n_classifiers', 'Accuracy'])
        df_results.append(df_temp)
    
    # Concatenar resultados
    df_combined = pd.concat(df_results)
    df_combined['Média'] = df_combined.groupby(['Method', 'n_classifiers'])['Accuracy'].transform('mean')
    results[name] = df_combined
    print(f"Finalizado o processamento do DataFrame: {name}")

# Criar MultiIndex final
final_result = pd.DataFrame()
for name, df in results.items():
    multiindex = pd.MultiIndex.from_tuples(
        [(name, row['Method'], row['n_classifiers']) for _, row in df.iterrows()],
        names=['DataFrame', 'Split Method', 'n_classifiers']
    )
    df.index = multiindex
    df = df.drop(columns=['Method', 'n_classifiers'])
    final_result = pd.concat([final_result, df])

# Salvar o resultado final em Excel
final_result.to_excel('voting_heterogeneo_resultados.xlsx', sheet_name='Resultados', index=True)


Iniciando processamento do DataFrame: hog_128_16
Realizando Holdout 70/30 para o DataFrame: hog_128_16
    Holdout: Voting com 5 classificadores base para hog_128_16
    Holdout: Voting com 10 classificadores base para hog_128_16
    Holdout: Voting com 15 classificadores base para hog_128_16
    Holdout: Voting com 20 classificadores base para hog_128_16
Realizando K-Fold Cross-Validation para o DataFrame: hog_128_16
    K-Fold: Voting com 5 classificadores base para hog_128_16


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

    K-Fold: Voting com 10 classificadores base para hog_128_16


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

    K-Fold: Voting com 15 classificadores base para hog_128_16


...

c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

    K-Fold: Voting com 20 classificadores base para cnn_VGG19_MAX_128_PCA


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

Finalizado o processamento do DataFrame: cnn_VGG19_MAX_128_PCA


c:\Users\david\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
